<a href="https://colab.research.google.com/github/j0rge-20/Precip/blob/main/Analisis_predicciones_vs_historico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install -q tensorflow-gpu
!pip install pandasql

import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import pandasql as ps
import seaborn as sns
from sklearn.metrics import mean_absolute_error, mean_squared_error
from math import sqrt
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc

from sklearn.model_selection import train_test_split
from pandas.plotting import register_matplotlib_converters
from pandasql import sqldf
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Bidirectional
from keras.layers import Dropout
from google.colab import files

%matplotlib inline
%config InlineBackend.figure_format='retina'

register_matplotlib_converters()
sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 22, 10

#sql function def
pysqldf = lambda q: sqldf(q, globals())


#dataset historico
#@title Loading dataset
path = 'https://raw.githubusercontent.com/j0rge-20/Precip/main/DatasetFull_v2.csv'
dfh = pd.read_csv(path, parse_dates=['Fecha'], index_col="Fecha")


#dataset predicciones
path3 = "https://raw.githubusercontent.com/j0rge-20/Precip/main/Predicciones_V4.csv"
dfp = pd.read_csv(path3, parse_dates=['Fecha'],index_col="Fecha")
#dfp.round({"Precipitacion":2})
dfp['Precipitacion'] = dfp['Precipitacion'].round(2)

conditions = [dfp['Estacion']==2120540,dfp['Estacion']==2306507,dfp['Estacion']==2120557,dfp['Estacion']==2401519,dfp['Estacion']==2120562,dfp['Estacion']==2120548,dfp['Estacion']==2120559,dfp['Estacion']==2120561,dfp['Estacion']==2120629,dfp['Estacion']==2401513,dfp['Estacion']==2120639,dfp['Estacion']==2120630,dfp['Estacion']==2120646,dfp['Estacion']==2120641,dfp['Estacion']==2120644,dfp['Estacion']==2120647,dfp['Estacion']==2119514,dfp['Estacion']==2306517,dfp['Estacion']==2306516]
values = ['Nemocón','Pacho','Subachoque','Ubaté','Guatavita','Chocontá','Bogotá I','Sibaté','Facatativá','Simijaca','La Mesa II','Bogotá II','El Colegio','Apulo','Ricaurte','La Mesa I','Fusagasugá','Guaduas','La Vega']
dfp['Zona'] = np.select(conditions,values)



path4 = 'https://raw.githubusercontent.com/j0rge-20/Precip/main/Estaciones.csv'
dfe = pd.read_csv(path4)
#rename columun
dfe.rename(columns ={'estacion':'Estacion'}, inplace = True )




print('#### Revisar repetidos dataframe ##########')
print (sqldf('select Estacion from (select Anio,Mes,Estacion, count(1) as qty from dfh  group by Anio,Mes,Estacion) where qty > 1 group by Estacion'))

###########################################   Funciones   #####################################################################

#array estaciones
estaciones  = ['2120540','2306507','2120557','2401519','2120562','2120548','2120559','2120561','2120629','2401513','2120639','2120630','2120646','2120641','2120644','2120647','2119514','2306517','2306516'] 
valores_ini = [0,0,0,0,0,0,0,0,0,0,0,0,0] 
#meses
meses =['-','enero','febrero','marzo','abril','mayo','junio','julio','agosto','septiembre','octubre','noviembre','diciembre']


#### Revisar repetidos dataframe ##########
Empty DataFrame
Columns: [Estacion]
Index: []


In [ ]:
#################### definicion estacion #######################################
#seccion 10 años
tam=120

def hist(tam):
  adf=[]  
  for i in range(0,19):
      adf.append (sqldf('select Anio, Mes, Estacion, Precipitacion from dfh where Estacion = '+ estaciones[i] + '').sort_values(by=['Anio','Mes']).tail(tam))
  return adf

adf = hist(tam)
dfht = adf[0].append(adf[1]).append(adf[2]).append(adf[3]).append(adf[4]).append(adf[5]).append(adf[6]).append(adf[7]).append(adf[8]).append(adf[9]).append(adf[10]).append(adf[11]).append(adf[12]).append(adf[13]).append(adf[14]).append(adf[15]).append(adf[16]).append(adf[17]).append(adf[18])

In [ ]:
#### HISTORICO MESES
#umbral 
umbral_pre = 40

dfht['Precipitacion'] = round(dfht['Precipitacion'],2) 
dfht['Agricultura'] = np.where(dfht['Precipitacion'] > umbral_pre ,1,0)

#190 x mes
#sqldf('select mes,count(*) from dfht group by mes')

dfhtc = (sqldf('select sum(Agricultura) as hit , Mes  from dfht  group by Mes  order by hit desc'))



conditions = [dfhtc['Mes']==1,dfhtc['Mes']==2,dfhtc['Mes']==3,dfhtc['Mes']==4,dfhtc['Mes']==5,dfhtc['Mes']==6,dfhtc['Mes']==7,dfhtc['Mes']==8,dfhtc['Mes']==9,dfhtc['Mes']==10,dfhtc['Mes']==11,dfhtc['Mes']==12]
values = ['enero','febrero','marzo','abril','mayo','junio','julio','agosto','septiembre','octubre','noviembre','diciembre']


dfhtc['Mes_Nombre'] = np.select(conditions,values)

dfhtcc = (sqldf('select Mes,Mes_Nombre,(hit*100)/190 as Porcentaje  from dfhtc  order by hit desc'))

dfhtcc



,Mes,Mes_Nombre,Porcentaje
0,4,abril,96
1,11,noviembre,93
2,5,mayo,91
3,3,marzo,90
4,10,octubre,90
5,2,febrero,66
6,12,diciembre,66
7,6,junio,63
8,9,septiembre,60
9,1,enero,53


In [ ]:
#predicciones1 MESES
#umbral 
umbral_pre = 40
dfp['Precipitacion'] = round(dfp['Precipitacion'],2) 
dfp['Agricultura'] = np.where(dfp['Precipitacion'] > umbral_pre ,1,0)

#19 x mes
#sqldf('select mes, count(*) from dfp group by mes')

dfpc = (sqldf('select sum(Agricultura) as hit , Mes  from dfp  group by Mes  order by hit desc'))

conditions = [dfpc['Mes']==1,dfpc['Mes']==2,dfpc['Mes']==3,dfpc['Mes']==4,dfpc['Mes']==5,dfpc['Mes']==6,dfpc['Mes']==7,dfpc['Mes']==8,dfpc['Mes']==9,dfpc['Mes']==10,dfpc['Mes']==11,dfpc['Mes']==12]
values = ['enero','febrero','marzo','abril','mayo','junio','julio','agosto','septiembre','octubre','noviembre','diciembre']


dfpc['Mes_Nombre'] = np.select(conditions,values)

dfpct = (sqldf('select Mes,Mes_Nombre,(hit*100)/19 as Porcentaje  from dfpc   order by hit desc'))

dfpct


,Mes,Mes_Nombre,Porcentaje
0,10,octubre,100
1,11,noviembre,100
2,5,mayo,89
3,4,abril,84
4,3,marzo,78
5,6,junio,78
6,9,septiembre,73
7,2,febrero,63
8,7,julio,52
9,12,diciembre,52


In [ ]:
#PREDICCIONES ESTACIONES

#umbral 
umbral_pre = 40
dfp['Precipitacion'] = round(dfp['Precipitacion'],2) 
dfp['Agricultura'] = np.where(dfp['Precipitacion'] > umbral_pre ,1,0)

dfpc = (sqldf('select Estacion, sum(Agricultura) as hit   from dfp  group by Estacion  order by hit desc'))

conditions = [dfpc['Estacion']==2120540,dfpc['Estacion']==2306507,dfpc['Estacion']==2120557,dfpc['Estacion']==2401519,dfpc['Estacion']==2120562,dfpc['Estacion']==2120548,dfpc['Estacion']==2120559,dfpc['Estacion']==2120561,dfpc['Estacion']==2120629,dfpc['Estacion']==2401513,dfpc['Estacion']==2120639,dfpc['Estacion']==2120630,dfpc['Estacion']==2120646,dfpc['Estacion']==2120641,dfpc['Estacion']==2120644,dfpc['Estacion']==2120647,dfpc['Estacion']==2119514,dfpc['Estacion']==2306517,dfpc['Estacion']==2306516]
values = ['Nemocón','Pacho','Subachoque','Ubaté','Guatavita','Chocontá','Bogotá I','Sibaté','Facatativá','Simijaca','La Mesa II','Bogotá II','El Colegio','Apulo','Ricaurte','La Mesa I','Fusagasugá','Guaduas','La Vega']

dfpc['Zona'] = np.select(conditions,values)

dfpc

dfpct = (sqldf('select Estacion,Zona,(hit*100)/12 as Porcentaje  from dfpc   order by hit desc'))

dfpct



,Estacion,Zona,Porcentaje
0,2120559,Bogotá I,91
1,2306516,La Vega,91
2,2120646,El Colegio,83
3,2120647,La Mesa I,83
4,2306507,Pacho,83
5,2306517,Guaduas,83
6,2119514,Fusagasugá,75
7,2120548,Chocontá,75
8,2120557,Subachoque,75
9,2120562,Guatavita,75


In [ ]:
#HISTORICO ESTACIONES

#umbral 
umbral_pre = 40
tam = 120

estaciones = ['2120540','2306507','2120557','2401519','2120562','2120548','2120559','2120561','2120629','2401513','2120639','2120630','2120646','2120641','2120644','2120647','2119514','2306517','2306516'] 



dfht['Precipitacion'] = round(dfht['Precipitacion'],2) 
dfht['Agricultura'] = np.where(dfht['Precipitacion'] > umbral_pre ,1,0)

#validate data for each station
#sqldf('select count(1), Estacion from dfht group by Estacion')

dfhtc = (sqldf('select Estacion, sum(Agricultura) as hit    from dfht  group by Estacion  order by hit desc'))


conditions = [dfhtc['Estacion']==2120540,dfhtc['Estacion']==2306507,dfhtc['Estacion']==2120557,dfhtc['Estacion']==2401519,dfhtc['Estacion']==2120562,dfhtc['Estacion']==2120548,dfhtc['Estacion']==2120559,dfhtc['Estacion']==2120561,dfhtc['Estacion']==2120629,dfhtc['Estacion']==2401513,dfhtc['Estacion']==2120639,dfhtc['Estacion']==2120630,dfhtc['Estacion']==2120646,dfhtc['Estacion']==2120641,dfhtc['Estacion']==2120644,dfhtc['Estacion']==2120647,dfhtc['Estacion']==2119514,dfhtc['Estacion']==2306517,dfhtc['Estacion']==2306516]
values = ['Nemocón','Pacho','Subachoque','Ubaté','Guatavita','Chocontá','Bogotá I','Sibaté','Facatativá','Simijaca','La Mesa II','Bogotá II','El Colegio','Apulo','Ricaurte','La Mesa I','Fusagasugá','Guaduas','La Vega']

dfhtc['Zona'] = np.select(conditions,values)


dfhtcc = (sqldf('select Estacion,Zona, (hit*100)/120  as Porcentaje  from dfhtc  order by hit desc'))

dfhtcc


,Estacion,Zona,Porcentaje
0,2120647,La Mesa I,88
1,2120646,El Colegio,85
2,2306516,La Vega,85
3,2119514,Fusagasugá,79
4,2120639,La Mesa II,79
5,2306517,Guaduas,79
6,2306507,Pacho,76
7,2120557,Subachoque,74
8,2120559,Bogotá I,73
9,2120629,Facatativá,72


In [ ]:
# identificación zonas altos niveles lluvia

#dfp
#precipitación maxima por mes
#sqldf('select mes,  max (precipitacion), estacion from dfp group by mes')

sqldf('select zona, estacion, mes, precipitacion from dfp where precipitacion > 200  order by mes')

,Zona,Estacion,Mes,Precipitacion
0,Pacho,2306507,2,245.75
1,La Vega,2306516,2,260.33
2,Pacho,2306507,3,200.48
3,La Mesa I,2120647,4,203.84
4,El Colegio,2120646,5,217.16
5,La Vega,2306516,5,240.01
6,Guaduas,2306517,5,340.63
7,La Mesa I,2120647,11,205.49
8,Pacho,2306507,11,222.17


In [ ]:
#join dataframes
#pd.merge(df1, df2, on="key")
#rankings_pd.rename(columns = {'test':'TEST'}, inplace = True) 
dft = pd.merge (dfe,dfp,on = "Estacion")

dftt = sqldf('select Anio, Mes, Estacion, ciudad as Ciudad,Precipitacion from dft')

dftt
name = 'Precipitaciones_2021_detalle'
dftt.to_csv(name + '.csv',index=False)
files.download( name + '.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>